# KHOA LUAN VIETSTOCK
Huynh Duc Huy

In [1]:
from __future__ import print_function
import time
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Input, merge, BatchNormalization,GRU
from keras.datasets import imdb

import os
from keras.preprocessing.text import Tokenizer

Using Theano backend.


In [2]:
max_features =20000
max_len = 200  # cut texts after this number of words (among top max_features most common words)

In [3]:
# get dataset and unzip: http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

X_train = []
y_train = []

path = 'C:/1_Research/Create_data/aclImdb/newest/vietstock_27_12/train/pos/'
X_train.extend([open(path + f).read() for f in os.listdir(path) if f.endswith('.txt')])
y_train.extend([1 for _ in range(1325)])

path = 'C:/1_Research/Create_data/aclImdb/newest/vietstock_27_12/train/neg/'
X_train.extend([open(path + f).read() for f in os.listdir(path) if f.endswith('.txt')])
y_train.extend([0 for _ in range(390)])

print('x:')
print(X_train[:1])
print(X_train[-1:])
print(len(X_train))
print('y:')
print(y_train[:1])
print(y_train[-1:])
print(len(y_train))

x:
[' trong t\xe1\xbb\x95ng_s\xe1\xbb\x91 m\xc3\xa3 c\xe1\xbb\x95_phi\xe1\xba\xbfu gi\xe1\xba\xa3m_gi\xc3\xa1 trong n\xc4\x83m m\xc3\xa3 gi\xe1\xba\xa3m m\xe1\xba\xa1nh nh\xe1\xba\xa5t v\xe1\xbb\x9bi bao_g\xe1\xbb\x93m svn gtt hla ptk dc avf asa hly hsi v\xc3\xa0 th nh\xc3\xb3m ng\xc3\xa0nh x\xc3\xa2y_d\xe1\xbb\xb1ng v\xc3\xa0 b\xe1\xba\xa5t_\xc4\x91\xe1\xbb\x99ng_s\xe1\xba\xa3n ng\xc3\xa0nh x\xc3\xa2y_d\xe1\xbb\xb1ng ng\xc3\xa0y ni\xc3\xaam_y\xe1\xba\xbft v\xe1\xbb\x91n \xc4\x91i\xe1\xbb\x81u_l\xe1\xbb\x87 \xc4\x91\xe1\xbb\x93ng gi\xc3\xa1 \xc4\x91\xc3\xa3 \xc4\x91i\xe1\xbb\x81u_ch\xe1\xbb\x89nh \xc4\x91\xe1\xba\xa7u n\xc4\x83m nbsp \xc4\x91\xe1\xbb\x93ng cpgi\xc3\xa1 t\xe1\xba\xa1i ng\xc3\xa0y phi\xc3\xaan cu\xe1\xbb\x91i n\xc4\x83m \xc4\x91\xe1\xbb\x93ng cp t\xe1\xbb\xb7_l\xe1\xbb\x87 gi\xe1\xba\xa3m kh\xe1\xbb\x91i_l\xc6\xb0\xe1\xbb\xa3ng giao_d\xe1\xbb\x8bch b\xc3\xacnh_qu\xc3\xa2n tu\xe1\xba\xa7n cpnh\xc3\xb3m ng\xc3\xa0nh v\xe1\xba\xadn_t\xe1\xba\xa3i v\xc3\xa0 kho_b\xc3\xa3i ng

In [4]:
# read in the test data

X_test = []
y_test = []

path = 'C:/1_Research/Create_data/aclImdb/newest/vietstock_27_12/test/pos/'
X_test.extend([open(path + f).read() for f in os.listdir(path) if f.endswith('.txt')])
y_test.extend([1 for _ in range(614)])

path = 'C:/1_Research/Create_data/aclImdb/newest/vietstock_27_12/test/neg/'
X_test.extend([open(path + f).read() for f in os.listdir(path) if f.endswith('.txt')])
y_test.extend([0 for _ in range(142)])

print('x:')
print(X_test[:1])
print(X_test[-1:])
print(len(X_test))
print('y:')
print(y_test[:1])
print(y_test[-1:])
print(len(y_test))

x:
[' hai chi s\xc3\xb4 k\xe1\xba\xbft phi\xc3\xaan t\xc4\x83ng \xc4\x91i\xe1\xbb\x83m vn index t\xc4\x83ng \xc4\x91i\xe1\xbb\x83m hay n\xe1\xba\xb1m \xe1\xbb\x9f hnx index t\xc6\xb0\xc6\xa1ng_t\xe1\xbb\xb1 t\xc4\x83ng \xc4\x91i\xe1\xbb\x83m hay n\xe1\xba\xb1m \xe1\xbb\x9f \xc4\x91i\xe1\xbb\x83m t\xe1\xbb\x95ng thanh kho\xe1\xba\xa3n tr\xc3\xaan c\xe1\xba\xa3 hai s\xc3\xa0n \xc4\x91\xe1\xba\xa1t tri\xe1\xbb\x87u \xc4\x91\xc6\xa1n_v\xe1\xbb\x8b \xe1\xbb\xa9ng v\xe1\xbb\x9bi t\xe1\xbb\xb7 \xc4\x91\xe1\xbb\x93ng nbsp vi\xe1\xbb\x87c t\xc4\x83ng \xc4\x91i\xe1\xbb\x83m ch\xe1\xbb\xa7_y\xe1\xba\xbfu \xc4\x91\xe1\xba\xbfn t\xe1\xbb\xab nh\xc3\xb3m large_cap khi ch\xe1\xbb\x89 c\xc3\xb3 nh\xc3\xb3m n\xc3\xa0y t\xc4\x83ng h\xc6\xa1n nbsp hai nh\xc3\xb3m mid_cap v\xc3\xa0 small_cap t\xc4\x83ng nh\xe1\xba\xb9 trong khi micro \xc4\x91\xe1\xbb\x8f \xc4\x91i\xe1\xbb\x83m v\xe1\xbb\x81 nh\xc3\xb3m ng\xc3\xa0nh s\xe1\xbb\x91 nh\xc3\xb3m t\xc4\x83ng \xc4\x91i\xe1\xbb\x83m l\xe1\xba\xa5n_\xc3\xa1p ho\xc

In [5]:
#tokenize works to list of integers where each integer is a key to a word
imdbTokenizer = Tokenizer(nb_words=max_features)

imdbTokenizer.fit_on_texts(X_train)

In [6]:
#print top 20 words 
#note zero is reserved for non frequent words
for word, value in imdbTokenizer.word_index.items():
    if value < 20:
        print(value, word)

2 trong
16 giá
11 cho
9 có
1 và
19 đã
14 đầu
3 đồng
13 công
8 các
18 được
5 của
12 tăng
4 năm
6 tỷ
15 doanh
17 cổ
7 với
10 là


In [7]:
#create int to word dictionary
intToWord = {}
for word, value in imdbTokenizer.word_index.items():
    intToWord[value] = word

#add a symbol for null placeholder
intToWord[0] = "!!!NA!!!"
    
print(intToWord[1])
print(intToWord[2])
print(intToWord[32])

và
trong
khi


In [8]:
#convert word strings to integer sequence lists
print(X_train[0])
print(imdbTokenizer.texts_to_sequences(X_train[:1]))
for value in imdbTokenizer.texts_to_sequences(X_train[:1])[0]:
    print(intToWord[value])
    
X_train = imdbTokenizer.texts_to_sequences(X_train)
X_test = imdbTokenizer.texts_to_sequences(X_test)

 trong tổng_số mã cổ_phiếu giảm_giá trong năm mã giảm mạnh nhất với bao_gồm svn gtt hla ptk dc avf asa hly hsi và th nhóm ngành xây_dựng và bất_động_sản ngành xây_dựng ngày niêm_yết vốn điều_lệ đồng giá đã điều_chỉnh đầu năm nbsp đồng cpgiá tại ngày phiên cuối năm đồng cp tỷ_lệ giảm khối_lượng giao_dịch bình_quân tuần cpnhóm ngành vận_tải và kho_bãi ngành vận_chuyển khách đường_bộ hệ_thống trạm dừng ngày niêm_yết vốn điều_lệ đồnggiá đã điều_chỉnh đầu năm đồng cpgiá tại ngày phiên cuối năm đồng cptỷ lệ giảm khối_lượng giao_dịch bình_quân tuần cpnhóm ngành sản_xuất ngành kim_loại và các sản_phẩm từ khoáng phi kim_loại ngày niêm_yết vốn điều_lệ đồng giá đã điều_chỉnh đầu năm đồng cpgiá tại ngày phiên cuối năm đồng cp tỷ_lệ giảm khối_lượng giao_dịch bình_quân tuần cpnhóm ngành khai_khoáng ngành khai_khoáng khác ngày niêm_yết vốn điều_lệ đồnggiá đã điều_chỉnh đầu năm đồng cpgiá tại ngày phiên cuối năm đồng cptỷ lệ giảm khối_lượng giao_dịch bình_quân tuần cpnhóm ngành xây_dựng và bất_động_sả

In [9]:
# Censor the data by having a max review length (in number of words)

#use this function to load data from keras pickle instead of munging as shown above
#(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features,
#                                                      test_split=0.2)

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print("Pad sequences (samples x time)")
X_train = sequence.pad_sequences(X_train, maxlen=max_len)
X_test = sequence.pad_sequences(X_test, maxlen=max_len)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
y_train = np.array(y_train)
y_test = np.array(y_test)

1715 train sequences
756 test sequences
Pad sequences (samples x time)
X_train shape: (1715L, 200L)
X_test shape: (756L, 200L)


In [10]:
#example of a sentence sequence, note that lower integers are words that occur more commonly
print("x:", X_train[0]) #per observation vector of 20000 words
print("y:", y_train[0]) #positive or negative review encoding

x: [ 394  397   50   75  152    3   16   19   75  337   14    4    3 2108   33
  117  141  214    4    3  241    6  152   24  260  110   85  102  199  427
  310 2301  161   34   69  161  246  436  913  217  117  394  397   50   75
  152 3583   19   75  337   14    4    3 2108   33  117  141  214    4    3
  241    6  152   24  260  110   85  102  199  427  310 2301  161   34   69
  161  641  502    1    8   34  217   23  904  936  641  502  117  394  397
   50   75  152 3583   19   75  337   14    4    3 2108   33  117  141  214
    4    3  241    6  152   24  260  110   85  102  199  427  310 2301  161
   34   69  161  246  436  913  217  117  394  397   50   75  152    3   16
   19   75  337   14    4    3 2108   33  117  141  214    4    3  241    6
  152   24  260  110   85  102  199  427  310 2301  161  213  360  161   87
 1238  117  394  397   50   75  152 3583   19   75  337   14    4  116    3
 2108   33  117  141  214    4    3  241    6  152   24  260  110   85  102
  199  42

In [11]:
# double check that word sequences behave/final dimensions are as expected
print("y distribution:", np.unique(y_train, return_counts=True))
print("max x word:", np.max(X_train), "; min x word", np.min(X_train))
print("y distribution test:", np.unique(y_test, return_counts=True))
print("max x word test:", np.max(X_test), "; min x word", np.min(X_test))

y distribution: (array([0, 1]), array([ 390, 1325], dtype=int64))
max x word: 8044 ; min x word 0
y distribution test: (array([0, 1]), array([142, 614], dtype=int64))
max x word test: 7990 ; min x word 0


In [12]:
print("most and least popular words: ")
print(np.unique(X_train, return_counts=True))
# as expected zero is the highly used word for words not in index

most and least popular words: 
(array([   0,    1,    2, ..., 8037, 8039, 8044]), array([11264,  4543,  3858, ...,     1,     1,     1], dtype=int64))


In [13]:
#set model hyper parameters
epochs = 6
embedding_neurons = 128
lstm_neurons = 64
batch_size =32

In [14]:
# Forward Pass LSTM Network

# this is the placeholder tensor for the input sequences
sequence = Input(shape=(max_len,), dtype='int32')
# this embedding layer will transform the sequences of integers
# into vectors of size embedding
# embedding layer converts dense int input to one-hot in real time to save memory
embedded = Embedding(max_features, embedding_neurons, input_length=max_len)(sequence)
# normalize embeddings by input/word in sentence
bnorm = BatchNormalization()(embedded)

# apply forwards LSTM layer size lstm_neurons
forwards = LSTM(lstm_neurons, dropout_W=0.2, dropout_U=0.2)(bnorm)

# dropout 
after_dp = Dropout(0.5)(forwards)
output = Dense(1, activation='sigmoid')(after_dp)

model_fdir_atom = Model(input=sequence, output=output)
# review model structure
print(model_fdir_atom.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 200)           0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 200, 128)      2560000     input_1[0][0]                    
____________________________________________________________________________________________________
batchnormalization_1 (BatchNormal(None, 200, 128)      256         embedding_1[0][0]                
____________________________________________________________________________________________________
lstm_1 (LSTM)                    (None, 64)            49408       batchnormalization_1[0][0]       
___________________________________________________________________________________________

In [15]:
# Forward pass LSTM network

# try using different optimizers and different optimizer configs
model_fdir_atom.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

print('Train...')
start_time = time.time()

history_fdir_atom = model_fdir_atom.fit(X_train, y_train,
                    batch_size=batch_size,
                    nb_epoch=epochs,
                    validation_data=[X_test, y_test], 
                    verbose=2)

end_time = time.time()
average_time_per_epoch = (end_time - start_time) / epochs
print("avg sec per epoch:", average_time_per_epoch)

Train...
Train on 1715 samples, validate on 756 samples
Epoch 1/6
23s - loss: 0.6147 - acc: 0.7003 - val_loss: 0.5095 - val_acc: 0.8095
Epoch 2/6
24s - loss: 0.4885 - acc: 0.7837 - val_loss: 0.5252 - val_acc: 0.7989
Epoch 3/6
24s - loss: 0.4109 - acc: 0.8163 - val_loss: 0.5569 - val_acc: 0.7870
Epoch 4/6
23s - loss: 0.3447 - acc: 0.8484 - val_loss: 0.6130 - val_acc: 0.7659
Epoch 5/6
22s - loss: 0.2766 - acc: 0.8810 - val_loss: 0.7153 - val_acc: 0.7235
Epoch 6/6
23s - loss: 0.2233 - acc: 0.9131 - val_loss: 0.7882 - val_acc: 0.7103
avg sec per epoch: 26.2056666613


In [16]:
# Bi-directional Atom

# based on keras tutorial: https://github.com/fchollet/keras/blob/master/examples/imdb_bidirectional_lstm.py

# this is the placeholder tensor for the input sequences
sequence = Input(shape=(max_len,), dtype='int32')
# this embedding layer will transform the sequences of integers
# into vectors of size embedding
# embedding layer converts dense int input to one-hot in real time to save memory
embedded = Embedding(max_features, embedding_neurons, input_length=max_len)(sequence)
# normalize embeddings by input/word in sentence
bnorm = BatchNormalization()(embedded)

# apply forwards LSTM layer size lstm_neurons
forwards = GRU(lstm_neurons, dropout_W=0.4, dropout_U=0.4)(bnorm)
# apply backwards LSTM
backwards = GRU(lstm_neurons, dropout_W=0.4, dropout_U=0.4, go_backwards=True)(bnorm)

# concatenate the outputs of the 2 LSTMs
merged = merge([forwards, backwards], mode='concat', concat_axis=-1)
after_dp = Dropout(0.5)(merged)
output = Dense(1, activation='sigmoid')(after_dp)

model_bidir_atom = Model(input=sequence, output=output)
# review model structure
print(model_bidir_atom.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 200)           0                                            
____________________________________________________________________________________________________
embedding_2 (Embedding)          (None, 200, 128)      2560000     input_2[0][0]                    
____________________________________________________________________________________________________
batchnormalization_2 (BatchNormal(None, 200, 128)      256         embedding_2[0][0]                
____________________________________________________________________________________________________
gru_1 (GRU)                      (None, 64)            37056       batchnormalization_2[0][0]       
___________________________________________________________________________________________

In [17]:
# Bi-directional Atom

# try using different optimizers and different optimizer configs
model_bidir_atom.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

print('Train...')
start_time = time.time()

history_bidir_atom = model_bidir_atom.fit(X_train, y_train,
                    batch_size=batch_size,
                    nb_epoch=epochs,
                    validation_data=[X_test, y_test], 
                    verbose=2)

end_time = time.time()
average_time_per_epoch = (end_time - start_time) / epochs
print("avg sec per epoch:", average_time_per_epoch)

Train...
Train on 1715 samples, validate on 756 samples
Epoch 1/6
31s - loss: 0.6697 - acc: 0.6426 - val_loss: 0.5182 - val_acc: 0.7923
Epoch 2/6
33s - loss: 0.5252 - acc: 0.7574 - val_loss: 0.5196 - val_acc: 0.7963
Epoch 3/6
31s - loss: 0.4650 - acc: 0.7918 - val_loss: 0.5324 - val_acc: 0.7831
Epoch 4/6
32s - loss: 0.4114 - acc: 0.8222 - val_loss: 0.5462 - val_acc: 0.7751
Epoch 5/6
33s - loss: 0.3743 - acc: 0.8367 - val_loss: 0.5722 - val_acc: 0.7738
Epoch 6/6
32s - loss: 0.3245 - acc: 0.8601 - val_loss: 0.6019 - val_acc: 0.7513
avg sec per epoch: 43.808499972
